# Coding a spam classifier with naive Bayes

### 1. Imports and pre-processing data

We load the data into a Pandas DataFrame, and then preprocess it by adding a string with the (non-repeated) lowercase words in the email.

In [1]:
import numpy as np

In [2]:
import pandas as pd
emails = pd.read_csv('emails.csv')

In [3]:
emails[:10]

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
5,"Subject: great nnews hello , welcome to medzo...",1
6,Subject: here ' s a hot play in motion homela...,1
7,Subject: save your money buy getting this thin...,1
8,Subject: undeliverable : home based business f...,1
9,Subject: save your money buy getting this thin...,1


In [4]:
def process_email(text):
    text = text.lower()
    return list(set(text.split()))

emails['words'] = emails['text'].apply(process_email)

In [5]:
emails[:10]

,text,spam,words
0,Subject: naturally irresistible your corporate...,1,"[not, your, without, effective, provide, three..."
1,Subject: the stock trading gunslinger fanny i...,1,"[not, pepperoni, subject:, hall, chisel, segov..."
2,Subject: unbelievable new homes made easy im ...,1,"[being, subject:, have, offer, your, advantage..."
3,Subject: 4 color printing special request add...,1,"[click, subject:, com, 5110, ), advertisement,..."
4,"Subject: do not have money , get software cds ...",1,"[me, not, subject:, have, tradgedies, old, fro..."
5,"Subject: great nnews hello , welcome to medzo...",1,"[subject:, have, ieading, andmanyother, tosher..."
6,Subject: here ' s a hot play in motion homela...,1,"[advantage, petrochemical, materia, wire, woui..."
7,Subject: save your money buy getting this thin...,1,"[not, ship, subject:, your, have, aicohol, jus..."
8,Subject: undeliverable : home based business f...,1,"[not, your, 27, sun, recognized, ,, grownups, ..."
9,Subject: save your money buy getting this thin...,1,"[not, ship, subject:, your, have, just, minute..."


In [6]:
num_emails = len(emails)
num_spam = sum(emails['spam'])

print("Number of emails:", num_emails)
print("Number of spam emails:", num_spam)
print()

# Calculating the prior probability that an email is spam
print("Probability of spam:", num_spam/num_emails)

Number of emails: 5728
Number of spam emails: 1368

Probability of spam: 0.2388268156424581


### 2. Training a naive Bayes model

Our plan is to write a dictionary, and in this dictionary record every word, and its pair of occurrences in spam and ham

In [7]:
model = {}

# Training process
for index, email in emails.iterrows():
    for word in email['words']:
        if word not in model:
            model[word] = {'spam': 1, 'ham': 1}
        if word in model:
            if email['spam']:
                model[word]['spam'] += 1
            else:
                model[word]['ham'] += 1

In [8]:
model['lottery']

{'spam': 9, 'ham': 1}

In [9]:
model['sale']

{'spam': 39, 'ham': 42}

### 3. Using the model to make predictions

In [10]:
def predict_bayes(word):
    word = word.lower()
    num_spam_with_word = model[word]['spam']
    num_ham_with_word = model[word]['ham']
    return 1.0*num_spam_with_word/(num_spam_with_word + num_ham_with_word)

In [11]:
predict_bayes('lottery')

0.9

In [12]:
predict_bayes('sale')

0.48148148148148145

In [16]:
def predict_naive_bayes(email):
    total = len(emails)
    num_spam = sum(emails['spam'])
    num_ham = total - num_spam
    email = email.lower()
    words = set(email.split())
    spams = [1.0]
    hams = [1.0]
    for word in words:
        if word in model:
            spams.append(model[word]['spam']/num_spam*total)
            hams.append(model[word]['ham']/num_ham*total)
    prod_spams = np.longdouble(np.prod(spams)*num_spam)
    prod_hams = np.longdouble(np.prod(hams)*num_ham)
    return prod_spams/(prod_spams + prod_hams)

In [17]:
predict_naive_bayes('lottery sale')

0.9638144470140116877

In [18]:
predict_naive_bayes('Hi mom how are you')

0.12554358867164464553

In [19]:
predict_naive_bayes('Hi MOM how aRe yoU afdjsaklfsdhgjasdhfjklsd')

0.12554358867164464553

In [20]:
predict_naive_bayes('meet me at the lobby of the hotel at nine am')

6.964603508395965419e-05

In [21]:
predict_naive_bayes('enter the lottery to win three million dollars')

0.99952342186774284207

In [22]:
predict_naive_bayes('buy cheap lottery easy money now')

0.99997347226596596756

In [23]:
predict_naive_bayes('Grokking Machine Learning by Luis Serrano')

0.4197107645484567334

In [24]:
predict_naive_bayes('asdfgh')

0.23882681564245810055